In [1]:
# 1- Basic parameters:
image_path = 'test_data/VEGFA_10ng.czi'
median_filtering = (7,7,7)
second_filtering = (5,5)
nuclei_median_filtering = (3,3,3)
# bead radius:
rad_ = 1

In [ ]:
# 2- Sprout segmentation:
# Importing necessary libraries and defining functions for the analysis:
import numpy as np
from czifile import CziFile
from skimage.filters import threshold_li
import scipy
import matplotlib.pyplot as plt
from skimage.measure import label   
from scipy import ndimage
from fil_finder import FilFinder2D
import astropy.units as u
from skimage.morphology import skeletonize as skeletonize_sci
import napari

def getLargestCC(segmentation):
    labels = label(segmentation)
    assert(labels.max() != 0)
    largestCC = labels == np.argmax(np.bincount(labels.flat)[1:])+1
    return largestCC

def modified_max_inscribed_circle(bw, f):    
    D = ndimage.distance_transform_edt(bw)
    Rs = -np.sort(-D, axis=None)
    R = Rs[0]
    RInds = np.argsort(-D, axis=None)
    RInds = RInds[Rs >= f*R]
    [cy, cx] = np.unravel_index(RInds, D.shape)
    return R, cx, cy

def create_circular_mask(h, w, center=None, radius=None):
    if center is None: 
        center = (int(w/2), int(h/2))
    if radius is None: 
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

with CziFile(image_path) as im:
    image_array = im.asarray()

# checking the shape of the array
print(image_array.shape)

# defining the correct channels into new arrays based on the above (3) shape
im_sprout = image_array[0,0,0,0,:,:,:,0]
im_nuclei = image_array[0,0,0,1,:,:,:,0]

# Opening napari
viewer = napari.Viewer()
viewer.add_image(im_sprout, name='im_sprout')

# median filtering the sprout channel
med_sprout = im_sprout
med_sprout = scipy.ndimage.median_filter(med_sprout, size=median_filtering)

viewer.add_image(med_sprout, name="median_filtered_sprout")    

# taking the maximum projection of the sprout channel and checking if median filtering size is ok
max_proj_0 = np.max(im_sprout, axis=0)
viewer.add_image(max_proj_0, name="max_projection_sprout") 

max_projection = np.max(med_sprout, axis=0)
viewer.add_image(max_projection, name="med_filtered_max_proj")  

# thresholding the max projection and filling the holes
from skimage.filters import threshold_otsu
bw_mproj = max_projection >= threshold_otsu(max_projection)
viewer.add_image(bw_mproj, name="threshold_otsu")  

largest_cc = getLargestCC(bw_mproj)
viewer.add_image(largest_cc, name="largest_connected_component")  

bw = ndimage.binary_fill_holes(largest_cc) * 1
viewer.add_image(bw, name="fill_holes")  

# median filtering to make the sprouts smoother. 
# If the sprouts are not smooth, the analysis might find the skeleton wrong.
med_bw = scipy.ndimage.median_filter(bw, size=second_filtering)
viewer.add_image(med_bw, name="median_filtering")

skeleton_sci = skeletonize_sci(med_bw)
# Check bw, med_bw, skeleton images to see if you need to change the size of median filtering above, before passing this part. 
viewer.add_labels(skeleton_sci, name="skeletonized")

# Fil-finder analysis of skeletons
# As long as it shows the branches, it is ok to continue with possible errors.
fil = FilFinder2D(med_bw, ang_scale=0.1*u.deg, beamwidth=1*u.arcsec, mask=med_bw)
fil.preprocess_image(flatten_percent=90)
fil.medskel(verbose=True)
fil.analyze_skeletons(skel_thresh=10*u.pix, prune_criteria="length")
branch_tables = fil.branch_tables()

# defining the sprout area, bead and labeling the sprouts
bb = fil.filament_extents
num_skl = fil.branch_properties['number'].squeeze()
pix_skl = fil.branch_properties['pixels'][0]
pix_skl = [x+bb[0][0] for x in pix_skl]

R, cx, cy = modified_max_inscribed_circle(bw, f=0.9)
sz = bw.shape
bead = np.zeros(sz, dtype=bool)
if len(cx) > 200: sp = 10
else: sp = 20 
for i in range(0, len(cx), sp):
    # You can change radius by changing rad_ value in the beginning to get the correct seperation of sprouts
    circ_ = create_circular_mask(sz[0], sz[1], center=(cx[i],cy[i]), radius=R*rad_)
    bead = np.logical_or(bead, circ_)
viewer.add_image(bead, name="bead")

bw_rmv_bead = np.logical_and(med_bw, np.logical_not(bead))
labels = label(bw_rmv_bead)
lbls = np.unique(labels[(fil.skeleton).astype(np.bool_)])
lbls_inx = np.nonzero(lbls)[0]

L = 1
sprouts = np.zeros(sz, dtype=np.uint8)
for i in lbls_inx:
    sprouts[labels==lbls[i]] = L
    L += 1
viewer.add_labels(sprouts, name="sprouts")

In [ ]:
# 3- Nuclei segmentation
# importing the necessary libraries, median filtering the nuclei array and creating a maximum projection image
from skimage.transform import resize
from skimage.segmentation import morphological_chan_vese, checkerboard_level_set
from skimage.measure import label, regionprops
from scipy.ndimage import gaussian_filter
from skimage.morphology import local_maxima
import skimage.graph

def shortest_path(bw, st_p, end_p):         
    costs = np.where(bw, 1, 1000)
    path, cost = skimage.graph.route_through_array(
        costs, start=st_p, end=end_p, fully_connected=True)
    return path, cost

viewer.add_image(im_nuclei, name="nuclei_channel")
med_nuclei = im_nuclei
for _ in range(2):
    med_nuclei = scipy.ndimage.median_filter(med_nuclei, size=nuclei_median_filtering)
viewer.add_image(med_nuclei, name="median_filtered_nuclei")

max_projection_n = np.max(med_nuclei, axis=0)
viewer.add_image(max_projection_n, name="max_proj_nuclei")

# Nuclei segmentation continues, omitting bead part (bead2) and other beads/migrated cells (bw_not)
bead2 = np.zeros(sz, dtype=bool)
bw_not = np.logical_not(bw)

if len(cx) > 200: sp = 10
else: sp = 20 
for i in range(0, len(cx), sp):
    # You can change radius to R*n in order to get the correct seperation of sprouts
    circ_2 = create_circular_mask(sz[0], sz[1], center=(cx[i],cy[i]), radius=R*1)
    bead2 = np.logical_or(bead2, circ_2)


med_nuclei2 = med_nuclei.copy()
for i in range(med_nuclei.shape[0]):
    med_nuclei2[i,:,:][np.where(bead2)] = 0
    med_nuclei2[i,:,:][np.where(bw_not)] = 0
    #np.logical_and(med_nuclei2[i,:,:], np.logical_not(bead2))

#viewer.add_image(med_nuclei2, name="med_nuclei2")

# Downsize the image if it is large, so that the segmentation will not take so much time depending on the computing power.
if med_nuclei.shape[1] >= 2048:
    md = 4
elif med_nuclei.shape[1] == 1024:
    md = 2
else:
    md = 1
med_n_ds = (med_nuclei2[:, ::md, ::md]).astype(np.float64)

# Chan-Vese segmentation:
# increasing/decreasing iterations for more precise detection of nuclei.
# Decrease iterations if not enough nuclei segmented. For details you can check scikit image chan-vese segmentation.
max_proj_med_nuclei2 = np.max(med_nuclei2, axis=0)
mask_ = np.zeros(med_nuclei2.shape)
for i in range(med_nuclei2.shape[0]):
    mask_[i,:,:] = max_proj_med_nuclei2
mask = (mask_[:, ::md, ::md]).astype(np.float64)

bw_n = morphological_chan_vese(image=med_n_ds, num_iter=10, init_level_set=mask, lambda1=1, lambda2=1) 

# Histogram of objects. To select the area threshold and correct object afterwards.
bw_nn = bw_n.astype(np.bool_)
bw_nn = (resize(bw_nn, output_shape=(med_nuclei.shape), order=0)).astype(np.bool_)
viewer.add_labels(bw_nn, name="Chan-Vese_segmentation")

# if satisfied continue. 
# It would be good if segmentation covers all the nuclei and you observe a seperation between nuclei groups.
# We will seperate the individual nucleus soon.

lbl_n = label(bw_nn, connectivity=1)
stats = regionprops(lbl_n)
label_n, area_n, eqds_n, maj_axis_n, box_n, centroid_n, image_n = [], [], [], [], [], [], []
for p in stats:
    label_n.append(p.label)
    area_n.append(p.area)
    eqds_n.append(p.equivalent_diameter)
    maj_axis_n.append(p.major_axis_length)
    box_n.append(p.bbox)
    centroid_n.append(p.centroid)
    image_n.append(p.image)

objects_library = {"label_n":label_n, "image_n":image_n, "area_n":area_n,
                   "eqds_n":eqds_n, "maj_Axis_n":maj_axis_n, "box_n":box_n, "centroid_n":centroid_n}

# You can change the range of the histogram to see the details
plt.figure(), plt.hist(area_n, bins=200, range=(0,2000))

In [ ]:
# 4- Lumen analysis and results
# Nuclei location detection:
# Based on histogram, you can change area_threshold:
area_threshold = [30, 400]
correct_obj = (np.array(objects_library["area_n"])>area_threshold[0]) & (np.array(objects_library["area_n"])<area_threshold[1])
in_range_eqds = np.array(objects_library["eqds_n"])[correct_obj]
cents = np.array(objects_library["centroid_n"])[correct_obj]
cents_f = cents.astype(int)
rep = np.zeros(med_nuclei.shape, dtype=np.bool_)
for i in cents_f:    
    rep[tuple(i)]=True
large_obj = np.array(objects_library["area_n"])>=area_threshold[1]
intensity_thr = 0.1
theta_gd = np.mean(in_range_eqds)
himg = med_nuclei / np.max(med_nuclei)

for i in range(len(large_obj)):
    if large_obj[i]: 
        #print(i)           
        bb = objects_library["box_n"][i]
        D = himg[bb[0]:bb[3], bb[1]:bb[4], bb[2]:bb[5]]        
        B = bw_nn[bb[0]:bb[3], bb[1]:bb[4], bb[2]:bb[5]]
        Dfil = gaussian_filter(np.pad(D, [5]), sigma=0)
        Dfil = Dfil[5:-5, 5:-5, 5:-5]               
        reg_max = local_maxima(Dfil, connectivity=3, indices=False, allow_borders=False)
        reg_max = np.logical_and(reg_max, B)        
        reg_max_idx = np.argwhere(reg_max) 
        if len(reg_max_idx):      
            reg_max_idx = [x for x in reg_max_idx if Dfil[tuple(x)]>=intensity_thr]
            reg_max_value = [Dfil[tuple(x)] for x in reg_max_idx] 
            ranked = np.argsort(reg_max_value)[::-1]            
            reg_max_idx = [reg_max_idx[x] for x in ranked]
            flags = np.ones(len(reg_max_idx), dtype=bool)
            for j in range(len(reg_max_idx)):
                if flags[j]:
                    for k in range(j+1, len(reg_max_idx)):                                                                                         
                        path_seq, cost = shortest_path(B, reg_max_idx[j], reg_max_idx[k])
                        if cost<theta_gd:
                            flags[k] = False    
            reg_max_idx = np.array(reg_max_idx)[flags]
        for x in reg_max_idx:
            x = tuple((x[0]+bb[0], x[1]+bb[1], x[2]+bb[2]))
            rep[tuple(x)]=True
vis_rep = np.max(rep, axis=0)

#print(rep)

[rr, cc] = np.where(vis_rep)
#plt.figure(), plt.imshow(max_projection_n)
#plt.scatter(cc, rr, s=5, c='yellow')

dots = []
for i in range(len(rr)):
    dots.append((rr[i],cc[i]))

max_proj_nuclei = np.max(med_nuclei2, axis=0)
viewer.add_image(max_proj_nuclei, name="maximum_projection_nuclei")
viewer.add_points(dots, name="nuclei", size=5, face_color='yellow')

# Lumen space analysis: Regional width and paired nuclei distance analysis
from scipy.ndimage import distance_transform_edt as distance
from scipy.spatial.distance import cdist
def curve_parametrization(L):
    # not entirely correct
    s = L[0]
    dist = np.sum((L-s)**2, axis=1)
    max_inx = np.argmax(dist)
    s = L[max_inx]
    dist = np.sum((L-s)**2, axis=1)
    ranked = np.argsort(dist)[::-1]
    return L[ranked]
def point_in_lineside (points, curve):
    """d=(x−x1)(y2−y1)−(y−y1)(x2−x1), where [x,y] are points to assess,
    and [x1 y1], [x2 y2] form the line segment
    if d>0 or d<0 the point is in either side of the line"""
    dist_to_curve = cdist(points, curve, 'euclidean')
    proj = np.argmin(dist_to_curve, axis=1)
    ll = len(L)-1
    if ll in proj: proj[proj==ll] = ll-1
    proj_ep = proj+1
    d1 = L[proj_ep] - L[proj]
    d2 = rep_coord - L[proj]
    d = (d2[:,1] * d1[:,0]) - (d2[:,0] *  d1[:,1])
    return d

skeleton = pix_skl
skeleton = [curve_parametrization(L) for L in skeleton]
num_sp = sprouts.max()
num_skl = len(skeleton)
mproj = np.logical_or(bead, sprouts)
D = distance(mproj)
rep_2d = np.max(rep, axis=0)
mproj_n = np.max(med_nuclei, axis=0)
plt.imshow(np.max(med_nuclei, axis=0))
i_list = []
skel_len_list = []
tip_width_list = []
stalk_width_list = []
root_width_list = []
paired_dist_list = []

for i in range(1, num_sp+1):
    sprout_name = str(i)
    prop = {"sp_skeleton":[]}
    sp = sprouts==i    
    skl_intsc = np.zeros(num_skl)
    for j in range(len(skeleton)):
        L = skeleton[j]
        intsc = np.array([sp[tuple(x)] for x in L])
        skl_intsc[j] = intsc.sum()
    max_inx = np.argmax(skl_intsc)
    L = skeleton[max_inx]
    intsc = np.array([sprouts[tuple(x)] for x in L])
    non_z = intsc!=0
    L = L[non_z,:]
    if non_z[0]==False:
        L = np.flipud(L)
    prop = {"sp_skeleton": L}
    prop["sp_skel_len"] = len(L) #>>chnge
    thick = np.array([D[tuple(x)] for x in L])
    prop["thick"] = thick
    print(sprout_name,"skeleton length is:",len(L))
    skel_len_list.append(len(L))
    
    if len(L)>30:
        # You can change the section of the sprout the width measurement is made. 
        # For example tip_sp = int(0.30*len(L))... would measure the thickness 
        # a bit closer to the middle part compared to 0.25 part of the sprout 
        tip_sp = int(0.25*len(L)); tip_sp = range(tip_sp-2,tip_sp+2)
        mid_sp = int(0.50*len(L)); mid_sp = range(mid_sp-2,mid_sp+2)
        end_sp = int(0.75*len(L)); end_sp = range(end_sp-2,end_sp+2)
        prop["tip_width"] = np.mean(thick[tip_sp])*2
        prop["stalk_width"] = np.mean(thick[mid_sp])*2
        prop["root_width"] = np.mean(thick[end_sp])*2
        
        tip_width_list.append(prop["tip_width"])
        stalk_width_list.append(prop["stalk_width"])
        root_width_list.append(prop["root_width"])
        
    else:
        mid_sp = int(0.50*len(L)); mid_sp = range(mid_sp-1,mid_sp+1)
        prop["stalk_width"] = np.mean(thick[mid_sp])*2
        stalk_width_list.append(prop["stalk_width"])

    rep_sp = np.logical_and(sp, rep_2d)
    rep_coord = np.array(np.where(rep_sp)).T
    if len(rep_coord)==0:
        prop["paired_n"] = []
        prop["paired_dist"] = []
        continue
    d = point_in_lineside (rep_coord, L)
    rep1 = rep_coord[d>0]
    rep2 = rep_coord[d<0]
    if len(rep1)>len(rep2):
        rep1 = rep_coord[d<0]
        rep2 = rep_coord[d>0]
    dist_to_curve = cdist(rep1, L, 'euclidean')
    proj1 = L[np.argmin(dist_to_curve, axis=1)]
    dist_to_curve = cdist(rep2, L, 'euclidean')
    proj2 = L[np.argmin(dist_to_curve, axis=1)]
    pair_dist = cdist(proj1, proj2, 'euclidean') 

    if len(pair_dist) > 0:
        mv = np.min(pair_dist, axis=1)
        mi = np.argmin(pair_dist, axis=1)
        p = np.vstack((range(len(mi)), mi)).T
        a = np.unique(mi)
        paired_n = np.zeros((len(a), 2), dtype=int)
        paired_dist = np.zeros(len(a))
        plt.plot(L[:,1], L[:,0], "r")
        plt.plot(rep1[:,1], rep1[:,0], '.', markersize=1, color="yellow")
        plt.plot(rep2[:,1], rep2[:,0], '.', markersize=1, color="green")
        plt.plot(proj1[:,1], proj1[:,0], '.', markersize=1, color="blue")
        plt.plot(proj2[:,1], proj2[:,0], '.', markersize=1, color="magenta")
        plt.axis('equal')
    
        for ii in range(len(a)):
            bb = np.where(mi==a[ii])[0]
            if len(bb)==1:
                paired_n[ii,:] = p[bb, :]
            else:
                cc = np.argmin(mv[bb])
                paired_n[ii,:] = p[bb[cc], :]
            pp = paired_n[ii, :]
            x1 = np.expand_dims((rep1[pp[0]]), axis=1).T
            x2 = np.expand_dims((rep2[pp[1]]), axis=1).T
        
        # Putting a threshold value for paired distance according to width results:
            if cdist(x1, x2, 'euclidean') < (np.max(thick)*2):
                paired_dist[ii] = cdist(x1, x2, 'euclidean')
            else:
                continue
        
            # Putting a threshold value for paired distance according to width results:
            if paired_dist[ii] < (np.max(thick)*2):
                x = np.append(x1,x2, axis=0)
                plt.plot(x[:,1], x[:,0])
            else:
                continue

        prop["paired_n"] = paired_n
        prop["paired_dist"] = paired_dist
    
        paired_dist_in_a_sprout=[]
        for i in paired_dist:
            if i > 0:
                paired_dist_in_a_sprout.append(i)
        print(sprout_name,"paired_dist_in_this_sprout:",paired_dist_in_a_sprout)

        if len(paired_dist_in_a_sprout)>0:
            paired_dist_average = sum(paired_dist_in_a_sprout)/len(paired_dist_in_a_sprout)
            print(sprout_name,"paired_dist_average:",paired_dist_average)
            paired_dist_list.append(paired_dist_average)
        else:
            continue
    else:
        continue
    
    fig_xyz = plt.savefig('analyzed_image', dpi = 300)  
    i_list.append(str(i))

nuclei_count = np.count_nonzero(rep)

# Printing the measured results
print("skeleton_length_measurements:",skel_len_list)
print()
print("tip_width_measurements:", tip_width_list)
print("stalk_width_measurements:", stalk_width_list)
print("root_width_measurements:", root_width_list)
print()
print("paired_dist_measurements per sprout:", paired_dist_list)
print()
print("sprout_number:", len(skel_len_list))
print("skeleton_length_avg:",sum(skel_len_list)/len(skel_len_list))
print("nuclei number:",nuclei_count)
print()
print("tip_width_avg",sum(tip_width_list)/len(tip_width_list))
print("stalk_width_avg",sum(stalk_width_list)/len(stalk_width_list))
print("root_width_avg",sum(root_width_list)/len(root_width_list))
print()
if len(paired_dist_list)>0:
    print("paired_dist average per bead:",sum(paired_dist_list)/len(paired_dist_list))

In [ ]:
# 5- VOLUMETRIC ANALYSIS
med_n_sp = (med_sprout[:, ::2, ::2]).astype(np.float64)

# Creating a mask which does not include the bead part only includes the sprout part for segmentation
sprouts_2D = (resize(sprouts, output_shape=(med_n_sp.shape[1],med_n_sp.shape[2]), order=0)).astype(np.bool_)
med_n_sp_copy = med_n_sp.copy()
for i in range(med_n_sp_copy.shape[0]):
    med_n_sp_copy[i,:,:] = sprouts_2D

# Chan-Vese segmentation part. You can increase iterations if there are extra parts segmented
# or decrease iterations if there are some target parts not segmented
bw_sp = morphological_chan_vese(image=med_n_sp, num_iter=8, init_level_set=med_n_sp_copy, lambda1=1, lambda2=1)
sprout_3D = (resize(bw_sp, output_shape=(med_sprout.shape), order=0)).astype(np.bool_)
viewer.add_labels(sprout_3D)

volume = np.count_nonzero(sprout_3D)
print("volume:",volume)

In [6]:
# 6- If you want the results in excel file:
import xlsxwriter

# Creating the excel file, once.
workbook = xlsxwriter.Workbook('analysis_results.xlsx')
worksheet = workbook.add_worksheet()

# writing the data into excel sheet:
worksheet.write('B1', str("total skeleton length"))
worksheet.write('B2', sum(skel_len_list))

worksheet.write('C1', str("average skeleton length"))
worksheet.write('C2', sum(skel_len_list)/len(skel_len_list))

worksheet.write('D1', str("sprout number"))
worksheet.write('D2', len(skel_len_list))

worksheet.write('E1', str("nuclei number"))
worksheet.write('E2', nuclei_count)

worksheet.write('F1', str("tip width"))
worksheet.write('F2', sum(tip_width_list)/len(tip_width_list))

worksheet.write('G1', str("stalk width"))
worksheet.write('G2', sum(stalk_width_list)/len(stalk_width_list))

worksheet.write('H1', str("root width"))
worksheet.write('H2', sum(root_width_list)/len(root_width_list))

worksheet.write('I1', str("sprout volume"))
worksheet.write('I2', volume)

worksheet.write('J1', str("average paired nuclei distance"))
if len(paired_dist_list)>0:
    worksheet.write('J2', sum(paired_dist_list)/len(paired_dist_list))

workbook.close()
print("analysis is done")